In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dudiw\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [ ]:
import os
filepath_recipes = os.path.abspath("C:/Users/zhan3/Desktop/RAW_recipes.csv")
filepath_interactions = os.path.abspath("C:/Users/zhan3/Desktop/RAW_interactions.csv")

In [ ]:
df_recipe = pd.read_csv(filepath_recipes)
df_interaction = pd.read_csv(filepath_interactions)
df = pd.merge(df_recipe, df_interaction, how='inner', left_on='id', right_on='recipe_id')

In [ ]:
df_rating = df.groupby('id').mean()['rating']

In [ ]:
df_reviews = df.groupby('id').count()['review']

In [ ]:
df_new = pd.merge(df_recipe, df_rating, how='left', on='id')

In [ ]:
df_new = df_new[['id', 'name', 'tags', 'description', 'ingredients', 'rating']]
df_new.dropna(axis=0, inplace=True)
df_final = df_new.reset_index(drop=True)

In [ ]:
df_final.head()

In [2]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

#### Run this cell carefully, it will take 30 minutes.

In [ ]:
# for i in range(len(df_final)):
    
#     ingredients = df_final.loc[i]['ingredients']
#     ingredients = re.sub('[^a-zA-Z]', ' ', ingredients)
#     name = df_final.loc[i]['name']
#     name = re.sub('[^a-zA-Z]', ' ', name)
#     combined = name + ingredients
#     combined = combined.split()
#     ps = PorterStemmer()
#     combined = [ps.stem(word) for word in combined if not word in set(stopwords.words('english'))]
#     combined = ' '.join(combined)
#     df_final.at[i, "combined words"] = combined    

In [ ]:
# df_final.to_csv('recipe.csv', index=False)

In [ ]:
# df_read = pd.read_csv('recipe.csv')
# df_read = pd.merge(df_read, df_reviews, how='left', on='id')
# df_words = df_read[['id', 'combined words', 'rating', 'review']]

In [ ]:
# df_words.to_csv('recipe_final.csv')

### NLP training

In [20]:
df_words = pd.read_csv("recipe_final.txt",index_col=[0])
df_words

,id,combined words,rating,review
0,137739,arriba bake winter squash mexican style winter...,5.0,3
1,31490,bit differ breakfast pizza prepar pizza crust ...,3.5,4
2,112140,kitchen chili ground beef yellow onion dice to...,4.0,1
3,59389,alouett potato spreadabl chees garlic herb new...,4.5,2
4,44061,amish tomato ketchup canning tomato juic appl ...,5.0,1
...,...,...,...,...
226652,486161,zydeco soup celeri onion green sweet pepper ga...,5.0,6
226653,493372,zydeco spice mix paprika salt garlic powder on...,5.0,1
226654,308080,zydeco ya ya devil egg hard cook egg mayonnais...,5.0,5
226655,298512,cooki design cooki stick butter eagl brand con...,1.0,1


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
# Map the word column into an array
text = df_words['combined words'].to_list()

# Calculate the TF-IDF matrix

model = TfidfVectorizer()
matrix = model.fit_transform(text)

In [23]:
def query_preprocessing(query):
    
    query = re.sub('[^a-zA-Z]', ' ', query)
    query = query.split()
    ps = PorterStemmer()
    query = [ps.stem(word) for word in query if not word in set(stopwords.words('english'))]
    query = ' '.join(query)
    
    return query

In [28]:
def get_recipe(query):
    
    query = query_preprocessing(query)
    query_matrix = model.transform([query])
    similarity = cosine_similarity(matrix, query_matrix)
    
    similarity.flatten()
    top10 = np.argsort(similarity.flatten())[-100:]
    length = len(top10)
#     print(sorted(similarity.flatten(), reverse=True))
    for i in range(length):
        j = top10[i]
        name = df_words.loc[j]['combined words']
        rating = df_words.loc[j]['rating']
        print(name)
        print(rating)
#     index = np.argmax(similarity)
#     name = df_read.loc[index]['name']
    
    return name

In [29]:
print(get_recipe('steam rice'))

jamaican curri chicken fri chicken veget oil garlic scallion pepper curri powder allspic ginger coconut milk potato steam rice mango chutney yogurt
2.6666666666666665
steam chocol cake granul sugar milk butter sweeten condens milk cocoa powder plain flour bake powder bake soda egg
5.0
steam chicken chicken wing tomato red chili ginger spring onion ginger juic rice wine light soya sauc fine sugar cornflour sesam oil garlic oil
5.0
pina colada shrimp shrimp coconut milk fresh lime juic pineappl juic dark rum fresh cilantro garlic clove salt black pepper steam rice
5.0
steam salmon pepper salmon red pepper green pepper red wine vinegar oliv oil garlic powder oregano salt
5.0
simpl steam cabbag simpli good green cabbag butter salt pepper
4.12
chines steam dumpl dip sauc flour egg salt water ground pork scallion fresh ginger pea chili pepper flake soy sauc rice vinegar honey sesam oil sesam seed
4.625
spici shrimp chile sauc sugar water canola oil red onion garlic clove larg shrimp serrano 

# consider drop the duplicates from name column